In [1]:
import sys

sys.path.append("../")

import pandas as pd
import numpy as np
import datetime
import os
from pprint import pprint
import matplotlib.pyplot as plt
import time
from vectorbtpro import *
from time import time
import helpers as pth
import platform
from dotenv import load_dotenv
import scipy.stats as stats
import time
import helpers as pth
from numba import njit
import talib

theme = "light"
vbt.settings.set_theme(theme)

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 20)
# plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (12, 7)
plt.rcParams["axes.formatter.useoffset"] = False
plt.rcParams["axes.formatter.limits"] = [-1000000000, 1000000000]
plt.style.use("classic" if theme == "light" else "dark_background")

if platform.system().lower() == "windows":
    base_data_path = "H:\\phitech-data\\01_raw"
else:
    from core_chains.simple.llm import make_Q_chain

    base_data_path = "../../phitech-data/01_raw"
    load_dotenv("../../sandatasci-core/credentials")
    Q = make_Q_chain("gpt-4o-instance1", __vsc_ipynb_file__)

In [2]:
%%html
<style>
.dataframe {
    font-size: 9pt; /* Adjust font size as needed */
}
</style>

In [3]:
symbols = ["MES", "6B"]
df = pth.SierraChartData.ffill(
    pth.SierraChartData.pull(
        symbols,
        timeframe="1min",
        start="2024-11-01",
        end="2024-12-01",
    )
)
df

c:\Users\spask\miniconda3\Lib\site-packages\vectorbtpro\data\base.py:1728: VBTWarning: Symbols have mismatching index. Setting missing data points to NaN.


In [4]:
close = df.data["MES"].close
close.vbt.plot()

FigureWidget({
    'data': [{'name': 'close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '8ae274f8-d8a5-44cf-bcac-b9472cc2be46',
              'x': array([datetime.datetime(2024, 11, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 2, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 12, 1, 23, 57, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 59, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([5808.5 , 5808.75, 5808.5 , ..., 6115.75, 6116.  , 6115.75])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'norm

### Portfolio

#### Buying

In [5]:
account_state = vbt.pf_enums.AccountState(
    cash=100.0,
    position=0.0,
    debt=0.0,
    locked_cash=0.0,
    free_cash=100.0,
)
account_state

AccountState(cash=100.0, position=0.0, debt=0.0, locked_cash=0.0, free_cash=100.0)

In [8]:
order_res, new_account_state = vbt.pf_nb.buy_nb(
    account_state,
    size=1.0,
    price=15.0,
)
order_res, new_account_state

(OrderResult(size=1.0, price=15.0, fees=0.0, side=0, status=0, status_info=-1),
 AccountState(cash=85.0, position=1.0, debt=0.0, locked_cash=0.0, free_cash=85.0))

In [21]:
vbt.pf_enums.OrderSide._fields[order_res.side], vbt.pf_enums.OrderStatus._fields[order_res.status]

('Buy', 'Filled')

In [24]:
order_res, new_account_state = vbt.pf_nb.buy_nb(
    account_state,
    size=np.inf,
    price=15.0,
    size_granularity=1,  # disable fractional transactions
    allow_partial=False,  # disable partial fills
)
order_res, new_account_state

(OrderResult(size=6.0, price=15.0, fees=0.0, side=0, status=0, status_info=-1),
 AccountState(cash=10.0, position=6.0, debt=0.0, locked_cash=0.0, free_cash=10.0))

In [39]:
order_res, new_account_state = vbt.pf_nb.buy_nb(
    account_state,
    size=np.inf,
    price=15.0,
    fees=0.01,
    slippage=0.01,
    percent=0.5,  # this means -> "use 50% of the cash available"
    allow_partial=False,
    size_granularity=1,
)
order_res, new_account_state

(OrderResult(size=3.0, price=15.15, fees=0.4545, side=0, status=0, status_info=-1),
 AccountState(cash=54.095499999999994, position=3.0, debt=0.0, locked_cash=0.0, free_cash=54.095499999999994))

In [32]:
price_area = vbt.pf_enums.PriceArea(
    open=10,
    high=14,
    low=8,
    close=12
)
order_result, new_account_state = vbt.pf_nb.buy_nb(
    account_state=account_state,
    size=np.inf,
    price=np.inf,  # this should throw an error !
    price_area=price_area,
    price_area_vio_mode=vbt.pf_enums.PriceAreaVioMode.Error
)

ValueError: Adjusted order price is above the highest price

#### Selling

In [34]:
account_state = vbt.pf_enums.AccountState(
    cash=0.0,
    position=10.0,
    debt=0.0,
    locked_cash=0.0,
    free_cash=0.0
)
order_result, new_account_state = vbt.pf_nb.sell_nb(
    account_state=account_state,
    size=2.0,
    price=15.0
)
order_result, new_account_state

(OrderResult(size=2.0, price=15.0, fees=0.0, side=1, status=0, status_info=-1),
 AccountState(cash=30.0, position=8.0, debt=0.0, locked_cash=0.0, free_cash=30.0))

In [63]:
# short!
account_state = vbt.pf_enums.AccountState(
    cash=100.0,
    position=0.0,
    debt=0.0,
    locked_cash=0.0,
    free_cash=100.0
)
order_result, new_account_state = vbt.pf_nb.sell_nb(
    account_state=account_state, 
    size=np.inf,  
    price=15.0,
    allow_partial=False,
    size_granularity=1,
)
order_result, new_account_state

(OrderResult(size=6.0, price=15.0, fees=0.0, side=1, status=0, status_info=-1),
 AccountState(cash=190.0, position=-6.0, debt=90.0, locked_cash=90.0, free_cash=10.0))

In [64]:
# reverse position
order_result, new_account_state = vbt.pf_nb.buy_nb(
    account_state=new_account_state, 
    size=7,  
    price=15.0,
    allow_partial=False,
    size_granularity=1,
)
order_result, new_account_state

(OrderResult(size=7.0, price=15.0, fees=0.0, side=0, status=0, status_info=-1),
 AccountState(cash=85.0, position=1.0, debt=0.0, locked_cash=0.0, free_cash=85.0))

#### Short with profit

In [109]:
# short!
account_state = vbt.pf_enums.AccountState(
    cash=100.0,
    position=0.0,
    debt=0.0,
    locked_cash=0.0,
    free_cash=100.0
)
order_result, new_account_state = vbt.pf_nb.sell_nb(
    account_state=account_state, 
    size=2,  
    price=15.0,
    allow_partial=False,
    size_granularity=1,
)
order_result, new_account_state

(OrderResult(size=2.0, price=15.0, fees=0.0, side=1, status=0, status_info=-1),
 AccountState(cash=130.0, position=-2.0, debt=30.0, locked_cash=30.0, free_cash=70.0))

In [110]:
# cover the short
order_result, new_account_state = vbt.pf_nb.buy_nb(
    account_state=new_account_state, 
    size=order_result.size,
    price=10.0,
    allow_partial=False,
    size_granularity=1,
)

print(f"moneyyy -> {new_account_state.free_cash}!")
order_result, new_account_state

moneyyy -> 110.0!


(OrderResult(size=2.0, price=10.0, fees=0.0, side=0, status=0, status_info=-1),
 AccountState(cash=110.0, position=0.0, debt=0.0, locked_cash=0.0, free_cash=110.0))

#### Closing positions

In [112]:
# long poisition
account_state = vbt.pf_enums.AccountState(
    cash=0.0,
    position=10.0,
    debt=0.0,
    locked_cash=0.0,
    free_cash=100.0
)
order_result, new_account_statem = vbt.pf_nb.long_sell_nb(
    account_state,
    size=np.inf,
    price=15.0,
)
order_result, new_account_state

(OrderResult(size=10.0, price=15.0, fees=0.0, side=1, status=0, status_info=-1),
 AccountState(cash=110.0, position=0.0, debt=0.0, locked_cash=0.0, free_cash=110.0))

#### Orders

In [155]:
exec_state = vbt.pf_enums.ExecState(
    cash=100.0,
    position=0.0,
    debt=0.0,
    locked_cash=0.0,
    free_cash=100.0,
    val_price=15.0,
    value=100.0
)

order = vbt.pf_nb.order_nb(1, 15, direction=2)
vbt.pprint(order)

print('---')
order_result, new_exec_state = vbt.pf_nb.execute_order_nb(exec_state, order)
vbt.pprint(order_result)
vbt.pprint(new_exec_state)

Order(
    size=1.0,
    price=15.0,
    size_type=0,
    direction=2,
    fees=0.0,
    fixed_fees=0.0,
    slippage=0.0,
    min_size=np.nan,
    max_size=np.nan,
    size_granularity=np.nan,
    leverage=1.0,
    leverage_mode=0,
    reject_prob=0.0,
    price_area_vio_mode=0,
    allow_partial=True,
    raise_reject=False,
    log=False
)
---
OrderResult(
    size=1.0,
    price=15.0,
    fees=0.0,
    side=0,
    status=0,
    status_info=-1
)
ExecState(
    cash=85.0,
    position=1.0,
    debt=0.0,
    locked_cash=0.0,
    free_cash=85.0,
    val_price=15.0,
    value=100.0
)


In [156]:
print('---')
order = vbt.pf_nb.order_nb(1, 15, direction=1)
vbt.pprint(order)

print('---')
order_result, new_exec_state = vbt.pf_nb.execute_order_nb(new_exec_state, order)
vbt.pprint(order_result)
vbt.pprint(new_exec_state)

---
Order(
    size=1.0,
    price=15.0,
    size_type=0,
    direction=1,
    fees=0.0,
    fixed_fees=0.0,
    slippage=0.0,
    min_size=np.nan,
    max_size=np.nan,
    size_granularity=np.nan,
    leverage=1.0,
    leverage_mode=0,
    reject_prob=0.0,
    price_area_vio_mode=0,
    allow_partial=True,
    raise_reject=False,
    log=False
)
---
OrderResult(
    size=1.0,
    price=15.0,
    fees=0.0,
    side=1,
    status=0,
    status_info=-1
)
ExecState(
    cash=100.0,
    position=0.0,
    debt=0.0,
    locked_cash=0.0,
    free_cash=100.0,
    val_price=15.0,
    value=100.0
)
